## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import gkey as g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_Database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Vostok,RU,46.4856,135.8833,70.72,85,42,2.62,scattered clouds
1,1,Bengkulu,ID,-3.8004,102.2655,83.77,68,80,6.96,broken clouds
2,2,Georgetown,MY,5.4112,100.3354,80.53,93,40,7.00,scattered clouds
3,3,Busselton,AU,-33.6500,115.3333,63.63,61,36,12.44,scattered clouds
4,4,Salalah,OM,17.0151,54.0924,77.09,94,100,8.05,overcast clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input(
      "What is the minimum temperature you would like for your trip? "))
max_temp = float(input(
      "What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[
                        (city_data_df["Max Temp"] <= max_temp) &
                        (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Bengkulu,ID,-3.8004,102.2655,83.77,68,80,6.96,broken clouds
2,2,Georgetown,MY,5.4112,100.3354,80.53,93,40,7.00,scattered clouds
4,4,Salalah,OM,17.0151,54.0924,77.09,94,100,8.05,overcast clouds
5,5,Pringsewu,ID,-5.3582,104.9744,84.72,72,82,3.91,broken clouds
8,8,Kahului,US,20.8947,-156.4700,78.82,70,20,18.41,few clouds
10,10,Caliling,PH,9.9804,122.4814,84.60,70,100,9.53,light rain
12,12,Cherskiy,RU,68.7500,161.3000,80.46,51,82,6.49,light rain
15,15,Jamestown,US,42.0970,-79.2353,78.39,68,100,9.22,overcast clouds
16,16,Atuona,PF,-9.8000,-139.0333,77.74,73,26,17.20,scattered clouds
17,17,Tual,ID,-5.6667,132.7500,79.48,79,100,17.87,overcast clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                269
City                   269
Country                269
Lat                    269
Lng                    269
Max Temp               269
Humidity               269
Cloudiness             269
Wind Speed             269
Current Description    269
dtype: int64

In [7]:
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [17]:
preferred_cities_df.dtypes

City_ID                  int64
City                    object
Country                 object
Lat                    float64
Lng                    float64
Max Temp               float64
Humidity                 int64
Cloudiness               int64
Wind Speed             float64
Current Description     object
dtype: object

In [18]:
preferred_cities_df[(preferred_cities_df["City"]=="")]

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description


In [19]:
preferred_cities_df[(preferred_cities_df["Country"]=="")]

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description


In [20]:
preferred_cities_df[(preferred_cities_df["Current Description"]=="")]

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description


In [21]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()

In [22]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Bengkulu,ID,83.77,broken clouds,-3.8004,102.2655,
2,Georgetown,MY,80.53,scattered clouds,5.4112,100.3354,
4,Salalah,OM,77.09,overcast clouds,17.0151,54.0924,
5,Pringsewu,ID,84.72,broken clouds,-5.3582,104.9744,
8,Kahului,US,78.82,few clouds,20.8947,-156.4700,
10,Caliling,PH,84.60,light rain,9.9804,122.4814,
12,Cherskiy,RU,80.46,light rain,68.7500,161.3000,
15,Jamestown,US,78.39,overcast clouds,42.0970,-79.2353,
16,Atuona,PF,77.74,scattered clouds,-9.8000,-139.0333,
17,Tual,ID,79.48,overcast clouds,-5.6667,132.7500,


In [23]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"    
    params["location"] = f"{lat},{lng}"
    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()   
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [24]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df[(hotel_df["Hotel Name"]!="")].copy()

In [33]:
clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Bengkulu,ID,83.77,broken clouds,-3.8004,102.2655,Grage Hotel Bengkulu
2,Georgetown,MY,80.53,scattered clouds,5.4112,100.3354,Cititel Penang
4,Salalah,OM,77.09,overcast clouds,17.0151,54.0924,"HAMDAN PLAZA HOTEL SALALAH, an HTG Hotel"
5,Pringsewu,ID,84.72,broken clouds,-5.3582,104.9744,Urban Style by Front One
8,Kahului,US,78.82,few clouds,20.8947,-156.4700,Maui Seaside Hotel
10,Caliling,PH,84.60,light rain,9.9804,122.4814,Camp Canaan
12,Cherskiy,RU,80.46,light rain,68.7500,161.3000,Gostinitsa
15,Jamestown,US,78.39,overcast clouds,42.0970,-79.2353,Hampton Inn & Suites Jamestown
16,Atuona,PF,77.74,scattered clouds,-9.8000,-139.0333,Villa Enata
17,Tual,ID,79.48,overcast clouds,-5.6667,132.7500,Grand Vilia Hotel


In [31]:
# 8a. Create the output File (CSV)
output_data_file = "../Vacation_Search/WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [29]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [30]:
# 11a. Add a marker layer for each city to the map. 
marker_layer = gmaps.marker_layer(locations,
                                 info_box_content=hotel_info)
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))